In [132]:
import sys
import re
!{sys.executable} -m pip install recordlinkage
import recordlinkage as rl
import numpy as np
import pandas as pd

In [133]:
df = pd.read_excel('1.xlsx', sheet_name = 'Лист1',  engine='openpyxl')
dfFCCP = pd.read_excel('2.xlsx', sheet_name = 'Лист 1', engine='openpyxl')

In [134]:
df.rename(columns={'Дата1': 'Дата2'}, inplace=True)
df['Взыскатель1'] = ''
dfFCCP['Взыскатель2'] = ''
def write_deptor(row, key, key_true):
    if row is not None:
        row[key] = str(row[key_true]).lower()
    return row

dfFCCP = dfFCCP.apply(lambda row: write_deptor(row,'Взыскатель2', 'Взыскатель'), axis=1)

keep = ['...']
filtered_dfFCCP = dfFCCP[ dfFCCP['Взыскатель2'].str.contains('| '.join(keep))].index

dfFCCP.loc[dfFCCP.index.isin(filtered_dfFCCP), 'Взыскатель2.1'] = '...'
dfFCCP.loc[~dfFCCP.index.isin(filtered_dfFCCP), 'Взыскатель2.1'] = '...'
dfFCCP['Взыскатель2.1'] = dfFCCP['Взыскатель2.1'].astype(str)

C:\Users\Nikita.GBURCRO\AppData\Local\Temp\ipykernel_12868\2923035830.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Оренбургский филиал АО "ЭнергосбыТ Плюс"' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfFCCP.loc[dfFCCP.index.isin(filtered_dfFCCP), 'Взыскатель2.1'] = 'Оренбургский филиал АО "ЭнергосбыТ Плюс"'


In [135]:
df['Должник1'] = ''
dfFCCP['Должник ф'] = ''
def write_deptor(row, key, key_true, key_false):
    if row is not None:
        row[key] = str(row[key_true]).lower()
    else:
        row[key] = str(row[key_false]).lower()
    return row
        
df = df.apply(lambda row: write_deptor(row,'Должник1', 'Ответчик', 'ФИО должника'), axis=1)
dfFCCP = dfFCCP.apply(lambda row: write_deptor(row,'Должник ф', 'Должник', 'Должник.1'), axis=1)

In [136]:
df["match1"] = df['Номер ИП'].apply(lambda x: re.search(r'(\d{5,6}/\d{2,4}/\d{5,6})(-ИП)', str(x)))
df["match2"] = df['Номер ИП'].apply(lambda x: re.search(r'(\d{5,6}/\d{2,4}/\d{5,6})', str(x)))

def matchIP(row):
    if row['match1'] is not None: 
        return row['match1'].group(1) + '-ИП'
    elif row['match2'] is not None:
        return row['match2'].group(0) + '-ИП'
    else:
        return None

df['ИП формат1']  = df.apply(matchIP, axis=1)
df = df.drop(['match1', 'match2'], axis=1)

In [137]:
df["match1"] = df['Номер'].apply(lambda x: re.search(r'(\d{1,2}-\d{3,4}/\d{2,3}/)(\d{2,4})', str(x)))
df["match2"] = df['Номер'].apply(lambda x: re.search(r'(\d{1,2}-\d{1,2}-\d{4,5}/)(\d{2,4})', str(x)))
df["match3"] = df['Номер'].apply(lambda x: re.search(r'(\D+)(\d{9})', str(x)))
df["match4"] = df['Номер'].apply(lambda x: re.search(r'(А47-\d{3,4}/\d{2,4})', str(x)))
# Проход по каждой строке и поиск номеров ИД формата


def matchID(row):
    if row['match1'] is not None:
        return row['match1'].group(1)
    elif row['match2'] is not None:
        return row['match2'].group(1)
    elif row['match3'] is not None:
        return row['match3'].group(2)
    elif row['match4'] is not None:
        return row['match4'].group(0)
    else: return None
    
df['Номер ИД формат1'] = df.apply(matchID, axis=1)
df = df.drop(['match1', 'match2', 'match3', 'match4'], axis=1)

In [148]:
dfFCCP["match1"] = dfFCCP['Регистрационный номер '].apply(lambda x: re.search(r'(\d{5,6}/\d{2,4}/\d{5,6})(-ИП)', str(x)))
dfFCCP["match2"] = dfFCCP['Регистрационный номер '].apply(lambda x: re.search(r'(\d{5,6}/\d{2,4}/\d{5,6})', str(x)))

dfFCCP['ИП формат2']  = dfFCCP.apply(matchIP, axis=1)
dfFCCP = dfFCCP.drop(['match1', 'match2'], axis=1)

dfFCCP["match1"] = dfFCCP['Номер ИД'].apply(lambda x: re.search(r'(\d{1,2}-\d{3,4}/\d{2,3}/)(\d{2,4})', str(x)))
dfFCCP["match2"] = dfFCCP['Номер ИД'].apply(lambda x: re.search(r'(\d{1,2}-\d{1,2}-\d{4,5}/)(\d{2,4})', str(x)))
dfFCCP["match3"] = dfFCCP['Номер ИД'].apply(lambda x: re.search(r'(\D+)(\d{9})', str(x)))
dfFCCP["match4"] = dfFCCP['Номер ИД'].apply(lambda x: re.search(r'(А47-\d{3,4}/\d{2,4})', str(x)))
# Проход по каждой строке и поиск номеров ИД формата

dfFCCP['Номер ИД формат2'] = dfFCCP.apply(matchID, axis=1)
dfFCCP = dfFCCP.drop(['match1', 'match2', 'match3', 'match4'], axis=1)

filtered_df292 = df[df["Статус ИП"].isin(["Возбуждено", "Подано"])& (~df['Поставщик (взыскатель)'].isna())].reset_index()
dfFCCP = dfFCCP.reset_index(drop=True)

In [147]:
indexer = rl.Index()
indexer.block(left_on=['Должник1', 'Номер ИД формат1', 'Поставщик (взыскатель)'], right_on=['Должник ф', 'Номер ИД формат2', 'Взыскатель2.1'])
debtor = indexer.index(filtered_df292, dfFCCP)
print(len(debtor))

compare = rl.Compare()
compare.exact('Должник1', 'Должник ф', label = 'Должник')
compare.exact('Номер ИД формат1', 'Номер ИД формат2', label = 'Номер ИД рез')
compare.exact('Поставщик (взыскатель)', 'Взыскатель2.1', label = 'Взыскатель')
features = compare.compute(debtor, filtered_df292, dfFCCP)
potential_matches = features[features.sum(axis=1) > 1].reset_index()

potential_matches['level_0'] = pd.to_numeric(potential_matches['level_0'])
potential_matches['level_1'] = pd.to_numeric(potential_matches['level_1'])
potential_matches['sum'] = potential_matches['Взыскатель']+potential_matches['Должник']+potential_matches['Номер ИД рез'] 
potential_matches['p0'] = potential_matches[potential_matches['Взыскатель'] is None, 'p0'] = potential_matches['level_1']

583


In [140]:
matches_index = potential_matches[['level_0', 'level_1', 'sum']]
non_matches = filtered_df292.copy().drop(matches_index['level_0'].array).reset_index().drop(['level_0'], axis=1)
non_fc_matches = dfFCCP.copy().drop(matches_index[matches_index['level_1'] != potential_matches['p0']]['level_1'].array).reset_index()

def get_matches(row, df_lev_0, df_lev_1 ):
    row292 = df_lev_0.iloc[row['level_0']]
    rowFCCP = df_lev_1.iloc[row['level_1']]
    return pd.Series(pd.concat([row292, rowFCCP]))

matches = matches_index.apply(lambda row: get_matches(row, filtered_df292, dfFCCP), axis=1).reset_index()

In [141]:
indexer = rl.Index()
indexer.block(left_on=['Должник1', 'ИП формат1'], right_on=['Должник ф', 'ИП формат2'])
debtor = indexer.index(non_matches, non_fc_matches)
len(debtor)

compare = rl.Compare()
compare.exact('Должник1', 'Должник ф', label = 'Должник')
compare.string('ИП формат1', 'ИП формат2', label = 'Номер ИП рез')
compare.exact('Поставщик (взыскатель)', 'Взыскатель2.1', label = 'Взыскатель')
features = compare.compute(debtor, non_matches, non_fc_matches)
potential_matches = features[features.sum(axis=1) > 1].reset_index(drop=True)

In [142]:
try:
    matches_index2 = potential_matches[['level_0', 'level_1']]
    non_fc_matches_all = non_fc_matches.drop(matches_index2['level_1'].array).reset_index()
    matches_index2, non_fc_matches_all = non_fc_matches_all.align(non_fc_matches_all, axis=1, copy=False)
    matches2 = matches_index2.apply(lambda row: get_matches(row, non_matches, dfFCCP), axis=1).reset_index()
    matches_all = pd.concat([matches, matches2], ignore_index=True)

except:
    non_fc_matches_all = non_fc_matches
    matches_all = matches_index.apply(lambda row: get_matches(row, filtered_df292, dfFCCP), axis=1).reset_index()
    
matches_all['Дата1'] = pd.to_datetime(matches_all['Дата1'], dayfirst=True).dt.strftime('%d.%m.%Y %H:%M:%S')
matches_all['Дата2'] = pd.to_datetime(matches_all['Дата2'], dayfirst=True).dt.strftime('%d.%m.%Y %H:%M:%S')    

In [143]:
matches_all['Дата1'] = matches_all['Дата1'].fillna('30.12.2050 00:00:00')
matches_all['Дата2'] = matches_all['Дата2'].fillna('30.12.2050 00:00:00')

In [144]:
matches_all_1 = matches_all[pd.to_datetime(matches_all['Дата1'], dayfirst=True) <= pd.to_datetime(matches_all['Дата2'], dayfirst=True)]
matches_all_2 = matches_all_1.loc[(matches_all_1.groupby(['Должник1', 'Номер ИД формат1'])['Дата подачи в суд']
                                                                                      .transform('max')  == matches_all_1['Дата1'])]
matches_all_3 = matches_all_2.loc[(matches_all_2.groupby(['Должник1', 'Номер ИД формат1'])['Дата подачи в суд']
                                                                                      .transform('max')  == matches_all_2['Дата1'])]
non_matches_f_1 = non_fc_matches_all.loc[(non_fc_matches_all['Статус'] =='Отказан') & (non_fc_matches_all.groupby(['Должник ф', 'Номер ИД формат2'])['Дата окончания']
                                                                                      .transform('max')  == non_fc_matches_all['Дата окончания'])]
non_matches_f_2 = non_matches_f_1.loc[(non_fc_matches_all['Статус'] =='Отказан') & (non_fc_matches_all.groupby(['Должник ф', 'ИП формат2'])['Дата окончания']
                                                                                      .transform('max')  == non_fc_matches_all['Дата окончания'])]                                                                                      
non_matches_f_3 = non_fc_matches_all.loc[(non_fc_matches_all['Статус']!='Отказан')&(non_fc_matches_all.groupby(['Должник ф', 'Номер ИД формат2'])['Дата возбуждения']
                                                                                      .transform('max')  == non_fc_matches_all['Дата 2'])]
non_matches_f_4 = non_matches_f_3.loc[(non_fc_matches_all['Статус']!='Отказан')&(non_fc_matches_all.groupby(['Должник ф', 'ИП формат2'])['Дата возбуждения']
                                                                                      .transform('max')  == non_fc_matches_all['Дата 2'])]
non_matches_f_5 = pd.concat([non_matches_f_2,non_matches_f_4], ignore_index=True)

In [145]:
non_matches2 = non_matches_f_5.loc[:, ~non_fc_matches_all.columns.isin(['index', 'ИП формат2', 'Номер ИД формат2', 'Должник ф', 'Взыскатель2', 'Взыскатель'])]
matches_all_3 = matches_all_1.loc[:, ~matches_all.columns.isin(['index', 'level_0', 'ИП формат2', 'ИП формат1',  'Номер ИД формат1',  'Номер ИД формат2', 'Должник1', 'Должник ф', 'Взыскатель2', 'Взыскатель'])]

In [146]:
with pd.ExcelWriter('результат.xlsx') as writer:
    pd.DataFrame(matches_all_3).to_excel(writer, sheet_name='Лист 1', index=False)
    pd.DataFrame(non_matches_f_5).to_excel(writer, sheet_name='Лист 2', index=False)